In [ ]:
import threading
import time
import random

thread_semaphore = threading.Semaphore(4)

def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

def threaded_quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left_part = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right_part = [x for x in arr if x > pivot]
    left_sorted = []
    right_sorted = []
    def sort_left():
        nonlocal left_sorted
        left_sorted = threaded_quicksort(left_part)
        thread_semaphore.release()
    def sort_right():
        nonlocal right_sorted
        right_sorted = threaded_quicksort(right_part)
        thread_semaphore.release()
    threads = []
    if len(left_part) > 100:
        thread_semaphore.acquire()
        t1 = threading.Thread(target=sort_left)
        t1.start()
        threads.append(t1)
    else:
        left_sorted = threaded_quicksort(left_part)
    if len(right_part) > 100:
        thread_semaphore.acquire()
        t2 = threading.Thread(target=sort_right)
        t2.start()
        threads.append(t2)
    else:
        right_sorted = threaded_quicksort(right_part)
    for t in threads:
        t.join()
    return left_sorted + middle + right_sorted

if __name__ == "__main__":
    data = [random.randint(1, 100000) for _ in range(10000)]
    start_time = time.time()
    quicksort(data.copy())
    print("Single-threaded quicksort time:", time.time() - start_time)
    start_time = time.time()
    threaded_quicksort(data.copy())
    print("Multi-threaded quicksort time:", time.time() - start_time)